# TO RUN
1. Make sure you installed all the dependencies
2. open **'init_notebook.py'** and change the **projectFolder** to the src folder of the repository.
3. In the following cell change the dataDir to the data directory of the InD dataset

In [1]:
exec(open('init_notebook.py').read())

current working dirE:\AV\DataSetExploration\drone-dataset-tools\src


In [2]:
%load_ext autoreload
%autoreload 2

import logging
logging.basicConfig(level=logging.INFO)
from loguru import logger
logger.remove()
logger.add(sys.stdout, format="{level} {message}", filter="my_module", level="INFO")

In [4]:
# Unit tests on preprocessed scene data of location 2

In [5]:
from extractors.LocationData import LocationData
scene10 = LocationData.load("../data/location-2", "2022-11-13-fps-2.5-scene-10.dll")

In [15]:
allPedDfs = scene10.getClippedPedDfs()
otherDfs = scene10.getClippedOtherDfs()
allPedIds = scene10.uniqueClippedPedIds()
allOtherIds = scene10.uniqueClippedOtherIds()

# pedClasses = allPedDfs['class'].unique()
otherClasses = otherDfs['class'].unique()

In [13]:
assert len(otherClasses) > 1
assert 'pedestrian' not in otherClasses

In [16]:
assert allPedIds == allPedDfs["uniqueTrackId"].unique()
assert allOtherIds == otherDfs["uniqueTrackId"].unique()

In [22]:
primaryPedId = allPedIds[0]
primaryDf = scene10.getPedDfByUniqueTrackId(primaryPedId).copy()
primaryPedId

18023

In [21]:
from extractors.PedScenarioBuilder import PedScenarioBuilder
scenarioBuilder = PedScenarioBuilder()

In [25]:
recordingId, start, end, roadWidth = scenarioBuilder.getRecordStartEndWidth(primaryDf)

In [26]:
recordingId, start, end, roadWidth

(18, 286, 1076, 5.0)

In [28]:
assert primaryDf["frame"].min() == start
assert primaryDf["frame"].max() == end